In [106]:
#from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels import regression,stats
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
from collections import deque
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from operator import itemgetter



In [107]:
df_rate = pd.read_csv("interest_rates.csv", index_col = 0).rename(columns = {"Date":'date'})
df_vix = pd.read_csv('vix.csv', index_col = 0).loc[:,['date','close']].rename(columns = {"close":'vix'})

In [108]:
df = pd.merge(df_rate, df_vix, how = 'left').iloc[15:,].reset_index().drop(columns = 'index').dropna()

In [109]:
df_recent = df.iloc[-14:, :]

In [110]:
df_recent

,date,rates_Aus,rates_US,rates_Japan,rates_HK,vix
1915,2021-03-30,0.1,0.0,-0.1,0.86,19.61
1916,2021-03-31,0.1,0.0,-0.1,0.86,19.40
1917,2021-04-01,0.1,0.0,-0.1,0.86,17.33
1921,2021-04-05,0.1,0.0,-0.1,0.86,17.91
1922,2021-04-06,0.1,0.0,-0.1,0.86,18.12
1923,2021-04-07,0.1,0.0,-0.1,0.86,17.16
1924,2021-04-08,0.1,0.0,-0.1,0.86,16.95
1925,2021-04-09,0.1,0.0,-0.1,0.86,16.69
1928,2021-04-12,0.1,0.0,-0.1,0.86,16.91
1929,2021-04-13,0.1,0.0,-0.1,0.86,16.65


In [111]:
df_recent = df.iloc[-14:, :]
np.array(df_recent.mean())
recent_measure =np.array(df_recent.mean())
df_dist = df.loc[:,['date']]
df_dist['dist'] = float('inf')
df_dist.reset_index(inplace = True, drop = True)
df.reset_index(inplace = True, drop = True)

In [112]:
def distance(arr1, arr2):
    return np.linalg.norm(arr1 - arr2)

for idx, row in df.iterrows():
    dist = distance(np.array(row)[1:], recent_measure)
    df_dist.at[idx, 'dist'] = dist


In [128]:
min_dist = float('inf')
start_date = ""
for idx, row in df_dist.iterrows():
    date_time_obj = datetime.strptime(row['date'], '%Y-%m-%d')
    if (idx + 252) < len(df_dist):
        df_sub = df_dist.iloc[idx: idx +252, : ]
        avg_dist = df_sub['dist'].mean()
        if avg_dist < min_dist:
            min_dist, start_date = avg_dist, row['date']

In [129]:
min_dist

4.163031658583156

In [130]:
start_date

'2016-02-17'

In [131]:
df_dist

,date,dist
0,2016-01-04,3.804792
1,2016-01-05,2.719325
2,2016-01-06,3.710277
3,2016-01-07,7.813679
4,2016-01-08,9.784233
...,...,...
1325,2021-04-13,0.766429
1326,2021-04-14,0.426429
1327,2021-04-15,0.846429
1328,2021-04-16,1.166429
